In [2]:
#change display for the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## Getting All Articles

In [24]:
all_links = pd.read_excel('all articles.xlsx', header = None)

In [25]:
all_links.columns = ['links']

In [26]:
all_links.tail()

,links
217,https://www.snaxshot.com/p/valentines-day-gift...
218,https://www.snaxshot.com/p/welcome-to-indoctri...
219,https://www.snaxshot.com/p/welcome-to-indoctri...
220,https://www.snaxshot.com/privacy?utm_source=
221,https://www.snaxshot.com/privacy?utm_source=us...


In [27]:
article_links = all_links.loc[all_links.links.str.contains(r'snaxshot.com/p/') & ~all_links.links.str.contains(r'comments')]

In [28]:
#we want to remove the first ever article as it doesn't really contain any content
article_links = article_links.drop(index = 29)

In [29]:
article_links.to_csv('art_links.csv', index = False)

## Selenium

In [44]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

### Testing a Single Element

In [52]:
# Initialize the Chrome webdriver
driver = webdriver.Chrome(ChromeDriverManager().install())

# Login credentials
username = "username"
password = "password"

# Navigate to the login page
login_url = r"https://substack.com/sign-in?redirect=%2F&for_pub=snaxshot"
driver.get(login_url)

# Find and click the button before entering the password
button = driver.find_element_by_class_name("login-option")
button.click()

# Give the browser some time to process the button click
time.sleep(2)

# Find the username and password input fields and enter your credentials
username_field = driver.find_element(By.NAME, "email")
password_field = driver.find_element(By.NAME, "password")

username_field.send_keys(username)
password_field.send_keys(password)

# Submit the login form
password_field.send_keys(Keys.ENTER)

# Give the browser some time to process the login request
time.sleep(3)

protected_url = article_links.links[1]
driver.get(protected_url)

#Find element
whole_text = driver.find_element_by_xpath("//*[@id='main']/div[2]/div/div[1]/div/article/div[4]/div[1]")

# Perform further processing or extraction as needed
print(whole_text.text)


Snaxshot is a newsletter on upcoming food and beverage trends that offers a curation of brands and aesthetics written by Andrea Hernández.
Our newsletter is read by biggest executives in food and beverage companies in the world, don’t miss out upgrade your subscription today!
Subscribed
🔮 Peek into the future:
Creating a Brave New Snax World
Snax at SXSW—partnering with Foxtrot and Showdrop
Snax at Museum of Ice Cream with Female Founder World
Oracular spectacular—what’s actually new?
Spoonful of news …open, fucking wide
Generation SNAX
Despite whatever other folks or publications may tout in their EXPO recaps that inundated LinkedIn this past week, there’s no functional beverage or palo santo brew that can help me recover from these past few weeks, not to mention the SVB fiasco that had me scrambling for an alternative bank account while heading out to Austin. Hope you missed me though—get those thumbs warmed up, they about to get a good cardio while scrolling through this issue….it’s

### Looping to Extract Relevant Data

In [54]:
# Initialize the Chrome webdriver
driver = webdriver.Chrome(ChromeDriverManager().install()) #options = options if you want non-default

# Login credentials
username = "username"
password = "password"

# Navigate to the login page
login_url = r"https://substack.com/sign-in?redirect=%2F&for_pub=snaxshot"
driver.get(login_url)

# Find and click the button before entering the password
button = driver.find_element_by_class_name("login-option")
button.click()

# Give the browser some time to process the button click
time.sleep(2)

# Find the username and password input fields and enter your credentials
username_field = driver.find_element(By.NAME, "email")
password_field = driver.find_element(By.NAME, "password")

username_field.send_keys(username)
password_field.send_keys(password)

# Submit the login form
password_field.send_keys(Keys.ENTER)

# Give the browser some time to process the login request
time.sleep(3)

titles = []
subtitles = []
dates = []
images = []
whole_texts = []
#start the loop of urls

for i in range(len(article_links)):
    driver.get(article_links.links[i])

#Find element
    title = driver.find_element(By.CLASS_NAME, "post-title.unpublished")
    
    sub = driver.find_element(By.CLASS_NAME, "subtitle")
    
    try: #tries one location
        date = driver.find_element(By.XPATH, "//*[@id='main']/div[2]/div/div[1]/div/article/div[2]/div/div[1]/div/div[2]/div[2]/div[1]")
    except: #tries another location if the first fails
        date = driver.find_element(By.XPATH, "//*[@id='main']/div[2]/div/div[1]/div/article/div[2]/div/div[1]/div/div/div/div[1]")
    
    image = driver.find_elements_by_xpath("//a//img")
    imageNames = []
    
    for element in imageLinks:
        imageNames.append(element.get_attribute("src"))
        
    whole_text = driver.find_element_by_xpath("//*[@id='main']/div[2]/div/div[1]/div/article/div[4]/div[1]")
    
    #creating the entry for the article
    titles.append(title.text.strip())
    subtitles.append(sub.text.strip())
    dates.append(date.text.strip())
    images.append(imageNames[2:-2])
    whole_texts.append(whole_text.text.strip())

In [55]:
whole_texts[0]

'A newsletter on upcoming food and beverage trends that offers a curation of brands and aesthetics written by Andrea Hernández.\nSnaxshot is ad-free as we are community funded, if you enjoy our content, contribute here. 🤗\n🔮 Peek into the future:\nBeauty is in the eye of the snaxboi\nOn origins of packaging\nAndy Warhol’s CPG influence\nRebrands as the new launch\nAround the world in Pantry Kiths\nOracular Spectacular and Trendspotting\nSpoonful of News\nCome on in, the water’s warm, you know you want to!\nSubscribed\nSitting Pretty, Oh So Pretty\nIn defense of aestheticism, our brains literally crave beauty. Studies have shown that “beauty” literally triggers a part of our cerebellum that controls our hand movement, making us literally want to reach out for beautiful things. Beauty is in the eye of the beholder, and it involves everything from art, mates or that adaptogenic milk cardboard at your local curated grocer, it is no surprise then, to see the current state of F&B, where ever

## Exporting Data Out

In [58]:
df = pd.DataFrame(list(zip(titles, subtitles, dates, images, whole_texts)), columns =['title', 'subtitle', 'date', 'images', 'text'])

In [60]:
df.to_csv('full_snax_07-23.csv')